In [1]:
!kaggle competitions download -c house-prices-advanced-regression-techniques


  0%|          | 0.00/199k [00:00<?, ?B/s]
100%|##########| 199k/199k [00:00<00:00, 2.17MB/s]


In [2]:
import zipfile

with zipfile.ZipFile('house-prices-advanced-regression-techniques.zip', 'r') as zip_ref:
    zip_ref.extractall('.')

In [3]:
import pandas as pd

In [4]:
raw_df = pd.read_csv("train.csv")

In [5]:
len(raw_df)

1460